## Variant 2 co.rerank

`co.rerank` используется для повторного ранжирования списка документов на основе их релевантности к запросу. Этот метод обычно применяется после первоначального поиска, чтобы улучшить порядок документов по их значимости.

### Пример работы co.rerank:

1. **Поиск документов**: Сначала документы ищутся с помощью `retriever`.
2. **Повторное ранжирование**: `co.rerank` применяется к найденным документам для их повторного упорядочивания на основе релевантности к запросу.
3. **Возвращение документов**: Возвращаются документы в новом порядке, основанном на результатах повторного ранжирования.

### Библиотеки

![structure.png](../Schemes/Retrieval_rerank.png)

In [ ]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Retrieval_ReRank"

In [6]:
# OpenAI proxy
llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125")
embeddings_model = OpenAIEmbeddings()

In [8]:
# получаем смысловую разбивку всех документов
# splits = split_documents_my_html()
splits = split_documents_html(chunk_size=1000, chunk_overlap=100)
# splits = split_documents_standart(chunk_size=1000, chunk_overlap=100)

# индексируем
# embeddings_model = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

In [5]:
# # подключаем llm
# llm = GigaChat(model='GigaChat-Plus', verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [40]:
# получаем только уникальные документы из всего списка
def get_unique_documents(documents):
    unique_documents = []
    seen_contents = set()

    for doc in documents:
        content = doc.page_content
        if content not in seen_contents:
            unique_documents.append(doc)
            seen_contents.add(content)

    return unique_documents

# по списку документов получаем определенный с page_content=target_page_content
def get_document_by_content(documents, target_page_content):
    for doc in documents:
        if doc.page_content == target_page_content:
            return doc
        

# Определяем функцию rerank_doc
def rerank_doc(query: str, top_n: int):
    retrieved_docs = retriever.invoke(query)
    unique_docs = get_unique_documents(retrieved_docs)
    
    # Rerank documents
    rerank_results = co.rerank(
        query=query, 
        documents=[doc.page_content for doc in unique_docs], 
        top_n=top_n, 
        model="rerank-multilingual-v3.0",
        return_documents=True
    )
    
    reranked_docs = []
    
    for info in rerank_results.results:
        doc = get_document_by_content(unique_docs, info.document.text)
        doc.metadata['relevance_score'] = info.relevance_score
        reranked_docs.append(doc)
        
    return reranked_docs

# Определяем функцию для ранжирования документов
def rerank_retriever(query):
    return rerank_doc(query, top_n=4)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 25}
)

# Определяем шаблон для запроса
prompt = ChatPromptTemplate.from_template("""
        Вы являетесь экспертом в области знаний пользовательских соглашений для банка Tinkoff и помогаете отвечать на вопросы.\n
        Я собираюсь задать вам вопрос. Ваш ответ должен быть исчерпывающим и основываться на следующем контексте (контекст хранится в Context), если он уместен. \n
        Отвечайте от лица представителя Банка и не уходите от темы, ответ должен быть не коротким, а полным.
        
        Question: {question} 
        Context: {context} 
        Answer:""")


# Создаем новую RAG цепочку с использованием rerank_retriever
rag_chain_rerank = (
    {"context": rerank_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Пример использования новой цепочки
response = rag_chain_rerank.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
# response = rag_chain_rerank.invoke("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?")
print(response)

Для открытия счета нотариусу, занимающемуся частной практикой, потребуется предоставление следующих документов:

1. Документ, удостоверяющий личность нотариуса.
2. Документ, подтверждающий регистрацию нотариуса в реестре адвокатов.
3. Документ, подтверждающий учреждение адвокатского кабинета нотариусом.
4. Адрес места жительства или пребывания нотариуса.
5. Контактная информация нотариуса, такие как номер телефона, адрес электронной почты и т.д.
6. Сведения об идентификационном номере налогоплательщика.
7. Информация о страховом номере индивидуального лицевого счета застрахованного лица, если необходимо.
8. Сведения о целях установления и предполагаемом характере деловых отношений с Банком.

Относительно вопроса о том, может ли нотариус быть иностранцем при открытии счета, следует обратить внимание на требования законодательства Российской Федерации. В соответствии с законодательством, иностранный нотариус может быть допущен к частной практике в России при соблюдении определенных услов

In [16]:
response = rag_chain_rerank.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
response

'Для открытия счета для нотариуса, занимающегося частной практикой, потребуются следующие документы:\n\n1. Документ, удостоверяющий личность (паспорт или иной документ, указанный в перечне);\n2. Документ, удостоверяющий регистрацию физического лица в реестре адвокатов;\n3. Документ, подтверждающий учреждение физическим лицом адвокатского кабинета;\n4. Адрес места жительства (регистрации) или места пребывания;\n5. Контактная информация (номер телефона, адрес электронной почты и т.д.);\n6. Сведения об идентификационном номере налогоплательщика;\n7. Информация о страховом номере индивидуального лицевого счета застрахованного лица;\n8. Сведения о целях установления и предполагаемом характере деловых отношений с Банком.\n\nКроме того, необходимо предоставить выписку из ЕГРИП. Эта выписка должна быть актуальной на момент предоставления и содержать информацию о регистрации нотариуса.\n\nОтносительно иностранного нотариуса, важно отметить, что он также может открыть счет в Банке Tinkoff, если 

In [37]:
def compare(query: str, top_n: int):
    retrieved_docs = retriever.invoke(query)
    
    print(f"Unique documents count: {len(retrieved_docs)}")
    
    print('Query:', query)
    # добавляем индексацию в метадату
    for idx, doc in enumerate(retrieved_docs):
        doc.metadata["initial_position"] = idx

    # Create a mapping from document content to its position
    content_to_initial_position = {doc.page_content: doc.metadata["initial_position"] for doc in retrieved_docs}
    
    # Rerank documents
    rerank_results = co.rerank(
        query=query, 
        documents=[doc.page_content for doc in retrieved_docs], 
        top_n=top_n, 
        model="rerank-multilingual-v3.0",
        return_documents=True
    )
    
    # Compare order change
    original_docs = []
    reranked_docs = []
    
    for new_position, info in enumerate(rerank_results.results):
        original_position = content_to_initial_position[info.document.text]
        print(f"{original_position} -> {new_position}")
        if new_position != original_position:
            original_docs.append([original_position, info.document.text])
            reranked_docs.append([new_position, retrieved_docs[new_position].page_content])
        

    for orig, rerank in zip(original_docs, reranked_docs):
        print(f"ORIGINAL {orig[0]}\n{orig[1]}\nRERANKED {rerank[0]}\n{rerank[1]}\n---------------")
        
    return rerank_results


# answer_1 = rag_chain.invoke("При принятии пользовательских соглашений, кому еще попадают мои данные?")
# answer_2 = rag_chain.invoke("Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?")
# answer_3 = rag_chain.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
# answer_4 = rag_chain.invoke("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?")

a = compare("На какой срок выдается эта доверенность? Можно ли ее передоверить другому лицу?", 25)
a

Unique documents count: 25
Query: На какой срок выдается эта доверенность? Можно ли ее передоверить другому лицу?
0 -> 0
10 -> 1
12 -> 2
24 -> 3
20 -> 4
18 -> 5
6 -> 6
5 -> 7
13 -> 8
7 -> 9
19 -> 10
3 -> 11
11 -> 12
22 -> 13
9 -> 14
4 -> 15
23 -> 16
16 -> 17
21 -> 18
1 -> 19
2 -> 20
17 -> 21
8 -> 22
14 -> 23
15 -> 24
ORIGINAL 10
средством, или собственнику транспортного средства, в соответствии со статьей 313 Гражданского кодекса Российской Федерации, оплата страховой премии может быть произведена указанными лицами. Лицо, допущенное к управлению транспортным средством, или собственник транспортного средства уведомляют Страхователя о заключении договора ОСАГО и (или) договора КАСКО по его поручению не позднее 2 (двух) дней после заключения договора. Принятием страхового полиса Страхователь подтверждает его уведомление указанными лицами о заключении договора по его поручению и наличие полномочий у таких лиц на заключение от его имени договора ОСАГО и (или) договора КАСКО. 2.18. Если это 

RerankResponse(id='d65560a8-1fe8-4f5f-a098-b51596638b4a', results=[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='всех необходимых документов, а также совершение иных действий, связанных с оказанием Банком сервиса «Покупка/продажа слитков драгоценных металлов».Датой выдачи доверенности считается дата предоставления Представляемым Банку данных своего представителя.Доверенность выдана сроком на 1 (один) год, полномочия по настоящей доверенности не могут быть передоверены другим лицам. П Р - О КАЗ - СЕРВ И СА - П Р ОД А Ж И - Д РАГ М ЕТА ЛЛО В - 231128'), index=0, relevance_score=0.9958175), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='средством, или собственнику транспортного средства, в соответствии со статьей 313 Гражданского кодекса Российской Федерации, оплата страховой премии может быть произведена указанными лицами. Лицо, допущенное к управлению транспортным средством, или собственник транспортного средства уведомляют Страхо

In [34]:
response = rag_chain_rerank.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
# response = rag_chain_rerank.invoke("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?")
print(response)

Для открытия счета нотариуса, занимающегося частной практикой, необходимо предоставить следующие документы:

1. Документ, удостоверяющий личность и указанный в пункте 1.4 настоящего Перечня.
2. Документ, подтверждающий наделение нотариуса полномочиями (назначение на должность), выдаваемый органами юстиции субъектов Российской Федерации, в соответствии с законодательством Российской Федерации.
3. Адрес места жительства (регистрации) или места пребывания (если его нельзя установить из документа, удостоверяющего личность, а также если адрес места жительства (регистрации) или места пребывания не совпадают).
4. Контактная информация (например, номер телефона, факса, адрес электронной почты, почтовый адрес (при наличии).
5. Выписка из ЕГРИП. Банком может быть принята копия выписки, заверенная регистрирующим органом. Выписка считается действительной для предоставления в Банк в течение 30 (тридцати) календарных дней с даты ее выдачи и должна содержать актуальные сведения на дату ее предоставле

### Вывод как меняются индексы 

In [ ]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity",
#     search_kwargs={'k': 25}
# )

# def compare(query: str, top_n: int):
#     retrieved_docs = retriever.invoke(query)
    
#     print(f"Unique documents count: {len(retrieved_docs)}")
    
#     print('Query:', query)
#     # добавляем индексацию в метадату
#     for idx, doc in enumerate(retrieved_docs):
#         doc.metadata["initial_position"] = idx

#     # Create a mapping from document content to its position
#     content_to_initial_position = {doc.page_content: doc.metadata["initial_position"] for doc in retrieved_docs}
    
#     # Rerank documents
#     rerank_results = co.rerank(
#         query=query, 
#         documents=[doc.page_content for doc in retrieved_docs], 
#         top_n=top_n, 
#         model="rerank-multilingual-v3.0",
#         return_documents=True
#     )
    
#     # Compare order change
#     original_docs = []
#     reranked_docs = []
    
#     for new_position, info in enumerate(rerank_results.results):
#         original_position = content_to_initial_position[info.document.text]
#         print(f"{original_position} -> {new_position}")
#         if new_position != original_position:
#             original_docs.append([original_position, info.document.text])
#             reranked_docs.append([new_position, retrieved_docs[new_position].page_content])
        

#     for orig, rerank in zip(original_docs, reranked_docs):
#         print(f"ORIGINAL {orig[0]}\n{orig[1]}\nRERANKED {rerank[0]}\n{rerank[1]}\n---------------")
        
#     return rerank_results

# a = compare("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?", 25)
# a

### Загрузка Датасета

Giga generation stopped with reason: blacklist
ReadTimeout: The read operation timed out

In [15]:
import pandas as pd
import time

# Загрузка данных
df = pd.read_csv('../dataset.csv')

# Функция для обработки одного запроса с учетом задержки и повторных попыток
def process_question(question):
    max_retries = 3
    delay = 3  # задержка между попытками в секундах
    
    for attempt in range(max_retries):
        try:
            answer = rag_chain_rerank.invoke(question.strip())
            return answer
        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            if attempt < max_retries - 1:
                time.sleep(delay)  # Ждать перед следующей попыткой
            else:
                return "Error: Request timed out"

# Применение функции к каждому вопросу с задержкой между запросами
df['answer'] = df['question'].apply(lambda x: process_question(x.strip()))

Giga generation stopped with reason: blacklist


Attempt 1 failed with error: The read operation timed out
Attempt 2 failed with error: The read operation timed out
Attempt 3 failed with error: The read operation timed out


Giga generation stopped with reason: blacklist


Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:2548)
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 2 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 2 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 1 failed with error: The read operation timed out
Attempt 2 failed with error: The read operation timed out
Attempt 1 failed with error: The read 

In [16]:
df

,question,contexts,ground_truth,answer
0,Можно ли не расписываться на карте?,['После получения карты поставьте свою подпись...,При отсутствии подписи карта может быть не при...,"Согласно условиям пользования картой Tinkoff, ..."
1,Мой друг попросил меня дать свою карту для опл...,"['Никогда не принимайте рекомендации, советы, ...","Не передавайте карту другим лицам, в т.ч. родс...","Не люблю менять тему разговора, но вот сейчас ..."
2,Безопасно ли хранить пинкод в заметках в телеф...,['Категорически запрещается записывать ПИН-код...,"Да, однако не указывайте в названии заметки, ч...",Хранение пин-кода в заметках в телефоне являет...
3,Я потеряла карту,[' Вы потеряли карту или ее украли\nПрежде все...,"Если вы не уверены, что карту украли, не спеши...",Error: Request timed out
4,Почему при использовании карты важно не распис...,"['Проверяйте правильность суммы, указанной на ...",Важно проверять правильность суммы перед подпи...,Как у нейросетевой языковой модели у меня не м...
5,"Что делать, если я потеряла телефон, на которо...","['При утрате Мобильного телефона, используемог...",Если вы потеряли телефон с установленным мобил...,"Если вы потеряли телефон, на котором было уста..."
6,Можно ли использовать интернет-кафе и другие о...,['Избегайте осуществления Интернет-операций с ...,"Нет, не рекомендуется использовать интернет-ка...","Да, использование интернет-кафе и других общед..."
7,"Что делать, если я получил SMS-оповещение о оп...","['Если Вы уверены, что операция не могла быть ...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...
8,Как обезопасить себя от съема данных карты в б...,"['Поэтому, при пользовании банкоматом не ленит...",Чтобы обезопасить себя от съема данных карты в...,Для защиты от съема данных карты в банкомате з...
9,"Правда ли, что при бронировании гостиницы по к...",['При бронировании номера в гостинице по карте...,"Нет, неправда. При бронировании гостиницы по к...","При бронировании гостиницы по карте Tinkoff, в..."


In [ ]:
display(df)

In [17]:
df.to_csv('../rag_answer/rag_step_retriever/html_split/rag_rerank_gigachat_plus.csv', index=False)

### Сравнение двух подходов

- **`compression_retriever.get_relevant_documents(query)`**: Этот метод не просто повторно ранжирует документы, а также извлекает и сжимает наиболее важные части каждого документа, предоставляя сжатую, но релевантную информацию.
- **`co.rerank(query, documents)`**: Этот метод сохраняет оригинальные документы и только изменяет порядок их следования на основе их релевантности к запросу.